In [1]:
!pip install -q gradio openai pypdf tiktoken langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.0 MB/s eta 0:00

In [3]:
import os

openai_key=""
os.environ['OPENAI_API_KEY']= openai_key

In [4]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import gradio as gr
import tiktoken
import openai

In [5]:
!wget https://s3.amazonaws.com/static.nomic.ai/gpt4all/2023_GPT4All_Technical_Report.pdf

--2023-09-19 11:54:55--  https://s3.amazonaws.com/static.nomic.ai/gpt4all/2023_GPT4All_Technical_Report.pdf
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.182.34.216, 52.216.85.149, 52.216.97.189, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.182.34.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3651423 (3.5M) [application/pdf]
Saving to: ‘2023_GPT4All_Technical_Report.pdf’

2023_GPT4All_Techni 100%[===================>]   3.48M  8.17MB/s    in 0.4s    

2023-09-19 11:54:55 (8.17 MB/s) - ‘2023_GPT4All_Technical_Report.pdf’ saved [3651423/3651423]



In [6]:
loader=PyPDFLoader("/content/2023_GPT4All_Technical_Report.pdf")

In [7]:
docs=loader.load()

In [9]:
# print(docs)

In [10]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [11]:
texts=text_splitter.split_documents(docs)

In [12]:
llm = OpenAI(temperature=0.5)

In [13]:
chain=load_summarize_chain(llm, chain_type='map_reduce', verbose=True)

In [14]:
chain.run(texts)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"GPT4All: Training an Assistant-style Chatbot with Large Scale Data
Distillation from GPT-3.5-Turbo
Yuvanesh Anand
yuvanesh@nomic.aiZach Nussbaum
zanussbaum@gmail.com
Brandon Duderstadt
brandon@nomic.aiBenjamin Schmidt
ben@nomic.aiAndriy Mulyar
andriy@nomic.ai
Abstract
This preliminary technical report describes the
development of GPT4All, a chatbot trained
over a massive curated corpus of assistant in-
teractions including word problems, story de-
scriptions, multi-turn dialogue, and code. We
openly release the collected data, data cura-
tion procedure, training code, and final model
weights to promote open research and repro-
ducibility. Additionally, we release quantized
4-bit versions of the model allowing virtually
anyone to run the model on CPU.
1 Data Collection and Curation
We collected roughly one million prompt-
response pai

' This paper presents the development of GPT4All, a chatbot trained on a large corpus of assistant interactions, including word problems, stories, multi-turn dialogue, and code. The data, training code, and model weights have been released for the community to build upon. The models were trained using LoRA, Stanford Alpaca, and LLAMA, and evaluated using human evaluation data and Self-Instruct. The data and training details have been released for research purposes, and commercial use is prohibited.'

In [16]:
def summarize_pdf(pdf_file_path):
  loader=PyPDFLoader(pdf_file_path)
  docs=loader.load()
  text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  texts=text_splitter.split_documents(docs)
  chain=load_summarize_chain(llm, chain_type="map_reduce")
  summary=chain.run(texts)
  return summary

In [17]:
input_pdf_path=gr.components.Textbox(label="Provide the PDF File Path ")
output_summary=gr.components.Textbox(label="Summary")

In [ ]:
interface=gr.Interface(
    fn=summarize_pdf,
    inputs=input_pdf_path,
    outputs=output_summary,
    title="PDF Summarizer",
    description="Provide PDF File Path to get the Summary"
).launch(share=True,debug=False)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ae0af83a13b0d8a3a3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
